SILVER LAYER (CLEANING)
What We Fix

Remove nulls

Fix datatypes

Remove negative quantities

In [0]:
# ---- Parameters (CI/CD compatible) ----
dbutils.widgets.text("storage_account", "retaildatalaketest")
dbutils.widgets.text("container", "ecommerce")

storage_account = dbutils.widgets.get("storage_account")
container = dbutils.widgets.get("container")

base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net"

In [0]:
from pyspark.sql.functions import col

# bronze_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/bronze/online_retail_bronze"
# silver_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/silver/online_retail_silver"

silver_path = f"{base_path}/silver/online_retail_silver"
bronze_path = f"{base_path}/bronze/online_retail_bronze"

# Read bronze
bronze_df = spark.read.format("delta").load(bronze_path)

# Clean and transform to silver
silver_df = (bronze_df
    # Ensure correct data types (in case inferSchema missed something)
    .withColumn("Quantity", col("Quantity").cast("int"))
    .withColumn("UnitPrice", col("Price").cast("double"))   # Usually column is "UnitPrice", not "Price"
    .withColumn("CustomerID", col("CustomerID").cast("bigint"))
    
    # Business filters
    .filter(col("Quantity") > 0)
    .filter(col("UnitPrice") > 0)
    .filter(col("CustomerID").isNotNull())   # dropna on CustomerID
    
    # Optional: Add derived columns
    .withColumn("TotalAmount", col("Quantity") * col("UnitPrice"))
)

# Show sample
silver_df.limit(10).display()

# Write to silver layer
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

spark.read.format("delta").load(silver_path).limit(10).display()
spark.sql("DESCRIBE HISTORY delta.`" + silver_path + "`").display()

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,UnitPrice,TotalAmount
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom,2.55,15.299999999999999
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom,2.75,22.0
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850,United Kingdom,7.65,15.3
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850,United Kingdom,4.25,25.5
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536368,22960,JAM MAKING SET WITH JARS,6,12/1/10 8:34,4.25,13047,United Kingdom,4.25,25.5


Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,UnitPrice,TotalAmount
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom,2.55,15.299999999999999
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom,2.75,22.0
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850,United Kingdom,7.65,15.3
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850,United Kingdom,4.25,25.5
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536368,22960,JAM MAKING SET WITH JARS,6,12/1/10 8:34,4.25,13047,United Kingdom,4.25,25.5


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2026-01-01T15:19:07.000Z,143258346942372,amar212b@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(3292093734080571),0101-142513-lkq78t8t-v2n,0,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 3572749, numDeletionVectorsRemoved -> 0, numOutputRows -> 397885, numOutputBytes -> 3572749)",null,Databricks-Runtime/17.3.x-photon-scala2.13
0,2026-01-01T15:18:20.000Z,143258346942372,amar212b@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(3292093734080571),0101-142513-lkq78t8t-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 397885, numOutputBytes -> 3572749)",null,Databricks-Runtime/17.3.x-photon-scala2.13


Choose or Create a Catalog and Schema
First, decide where to put them. Common setup:
Catalog: main (default) or create your own like ecommerce
Schema: bronze and silver
Run this SQL to create them if they don't exist:

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ecommerce
MANAGED LOCATION 'abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/managed/ecommerce/';

CREATE SCHEMA IF NOT EXISTS ecommerce.bronze;
CREATE SCHEMA IF NOT EXISTS ecommerce.silver;

Register the Silver,Bronze Table

In [0]:
%sql
-- Bronze table
CREATE TABLE IF NOT EXISTS ecommerce.bronze.online_retail_bronze
USING DELTA
LOCATION 'abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/bronze/online_retail_bronze';

-- Silver table
CREATE TABLE IF NOT EXISTS ecommerce.silver.online_retail_silver
USING DELTA
LOCATION 'abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/silver/online_retail_silver';

check if table exist and is querying

In [0]:
%sql
SELECT * FROM ecommerce.silver.online_retail_silver LIMIT 10

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,UnitPrice,TotalAmount
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom,2.55,15.299999999999999
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom,2.75,22.0
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,3.39,20.34
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850,United Kingdom,7.65,15.3
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850,United Kingdom,4.25,25.5
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom,1.85,11.100000000000001
536368,22960,JAM MAKING SET WITH JARS,6,12/1/10 8:34,4.25,13047,United Kingdom,4.25,25.5
